# LNDL Errors - Exception Hierarchy for Structured Output Parsing

LNDL (Lionherd Natural Data Language) defines a structured exception hierarchy for parsing and validation errors. These errors provide clear diagnostics when LLM responses don't conform to expected LNDL syntax.

**Core Features:**
- **Base Exception**: `LNDLError` - Catch all LNDL-related errors
- **Specific Error Types**: 6 specialized exceptions for different failure modes
- **Clear Diagnostics**: Descriptive error messages guide debugging
- **Validation Stages**: Errors map to specific parsing/validation phases

In [1]:
from pydantic import BaseModel

from lionpride.lndl import (
    AmbiguousMatchError,
    InvalidConstructorError,
    Lexer,
    LNDLError,
    MissingFieldError,
    MissingLvarError,
    MissingOutBlockError,
    Parser,
    TypeMismatchError,
)
from lionpride.lndl.resolver import parse_lndl
from lionpride.types import Operable, Spec

## 1. Error Hierarchy Overview

All LNDL errors inherit from `LNDLError`, enabling catch-all error handling.

In [2]:
# Check inheritance
print(f"MissingLvarError is LNDLError: {issubclass(MissingLvarError, LNDLError)}")
print(f"MissingFieldError is LNDLError: {issubclass(MissingFieldError, LNDLError)}")
print(f"TypeMismatchError is LNDLError: {issubclass(TypeMismatchError, LNDLError)}")
print(f"InvalidConstructorError is LNDLError: {issubclass(InvalidConstructorError, LNDLError)}")
print(f"MissingOutBlockError is LNDLError: {issubclass(MissingOutBlockError, LNDLError)}")
print(f"AmbiguousMatchError is LNDLError: {issubclass(AmbiguousMatchError, LNDLError)}")

# All are also standard Exceptions
print(f"\nLNDLError is Exception: {issubclass(LNDLError, Exception)}")

MissingLvarError is LNDLError: True
MissingFieldError is LNDLError: True
TypeMismatchError is LNDLError: True
InvalidConstructorError is LNDLError: True
MissingOutBlockError is LNDLError: True
AmbiguousMatchError is LNDLError: True

LNDLError is Exception: True


## 2. MissingOutBlockError

Raised when no `OUT{}` block found in LLM response. This is the first validation stage.

In [3]:
# Response missing OUT{} block
invalid_response = """
Here is my analysis:

The sentiment is positive and the confidence is high.
"""

try:
    # Parse using Lexer/Parser API
    lexer = Lexer(invalid_response)
    tokens = lexer.tokenize()
    parser = Parser(tokens, source_text=invalid_response)
    program = parser.parse()

    if not program.out_block:
        raise MissingOutBlockError("No OUT{} block found in response")
except MissingOutBlockError as e:
    print(f"✓ Caught MissingOutBlockError: {e}")
    print(f"  Type: {type(e).__name__}")

✓ Caught MissingOutBlockError: No OUT{} block found in response
  Type: MissingOutBlockError


In [4]:
# Unbalanced braces caught by Parser
unbalanced = """
OUT{
  sentiment: "positive",
  confidence: 0.95

"""

try:
    # Parser detects unbalanced braces during tokenization/parsing
    lexer = Lexer(unbalanced)
    tokens = lexer.tokenize()
    parser = Parser(tokens, source_text=unbalanced)
    program = parser.parse()
except Exception as e:
    # Parser raises ParseError for structural issues
    print(f"✓ Caught unbalanced braces: {type(e).__name__}: {e}")

## 3. MissingFieldError

Raised when a required Spec field is missing from the `OUT{}` block.

In [5]:
# Create operable with required specs (scalar types)
operable = Operable(
    specs=[
        Spec(str, name="sentiment", required=True),
        Spec(float, name="confidence", required=True),
        Spec(str, name="explanation", required=True),  # Missing in response
    ]
)

# Response missing 'explanation' field
incomplete_response = """
OUT{
  sentiment: "positive"
  confidence: 0.95
}
"""

try:
    parse_lndl(incomplete_response, operable)
except MissingFieldError as e:
    print(f"✓ Caught MissingFieldError: {e}")
    print("  Missing field: explanation")

✓ Caught MissingFieldError: Required field 'explanation' missing from OUT{}
  Missing field: explanation


In [6]:
# Optional fields don't raise MissingFieldError
operable_optional = Operable(
    specs=[
        Spec(str, name="sentiment", required=True),
        Spec(float, name="confidence", required=False),  # Optional
    ]
)

partial_response = """
OUT{
  sentiment: "positive"
}
"""

result = parse_lndl(partial_response, operable_optional)
print("✓ Optional field allowed to be missing")
print(f"  sentiment: {result.sentiment}")  # Attribute access
print(f"  confidence: {result.fields.get('confidence', 'NOT PROVIDED')}")

✓ Optional field allowed to be missing
  sentiment: positive
  confidence: NOT PROVIDED


## 4. MissingLvarError

Raised when `OUT{}` references an `<lvar>` that doesn't exist. This catches typos and missing variable declarations.

In [7]:
# Define BaseModel for array syntax
class Analysis(BaseModel):
    result: str


operable_lvar = Operable(
    specs=[Spec(Analysis, name="analysis")]  # BaseModel spec
)

# Reference non-existent lvar (typo: my_rezult instead of my_result)
typo_response = """
<lvar Analysis.result my_result>
The analysis shows positive trends.
</lvar>

OUT{
  analysis: [my_rezult]
}
"""

try:
    parse_lndl(typo_response, operable_lvar)
except ExceptionGroup as eg:
    # LNDL validation errors wrapped in ExceptionGroup
    print("✓ Caught missing lvar reference (ExceptionGroup)")
    for exc in eg.exceptions:
        if "my_rezult" in str(exc):
            print("  Variable 'my_rezult' not found (typo in 'my_result')")
except ValueError as e:
    # Fallback for direct ValueError
    print(f"✓ Caught missing lvar reference: {e}")

✓ Caught missing lvar reference (ExceptionGroup)
  Variable 'my_rezult' not found (typo in 'my_result')


## 5. TypeMismatchError

Raised when constructor class in `<lvar>` doesn't match the expected Spec type.

In [8]:
# Define two different models
class PersonInfo(BaseModel):
    name: str
    age: int


class CompanyInfo(BaseModel):
    name: str
    founded: int


# Operable expects PersonInfo
operable_person = Operable(specs=[Spec(PersonInfo, name="entity")])

# Response provides CompanyInfo instead
type_mismatch_response = """
<lvar CompanyInfo.name comp_name>Acme Corp</lvar>
<lvar CompanyInfo.founded comp_year>1985</lvar>

OUT{
  entity: [comp_name, comp_year]
}
"""

try:
    parse_lndl(type_mismatch_response, operable_person)
except ExceptionGroup as eg:
    # Validation errors wrapped in ExceptionGroup
    print("✓ Caught type mismatch error (ExceptionGroup)")
    for exc in eg.exceptions:
        if isinstance(exc, TypeMismatchError):
            print(f"  {exc}")
            print("  Expected: PersonInfo, Got: CompanyInfo")
except TypeMismatchError as e:
    # Fallback for direct error
    print(f"✓ Caught TypeMismatchError: {e}")
    print("  Expected: PersonInfo")
    print("  Got: CompanyInfo")

✓ Caught type mismatch error (ExceptionGroup)
  Variable 'comp_name' is for model 'CompanyInfo', but field 'entity' expects 'PersonInfo'
  Expected: PersonInfo, Got: CompanyInfo


## 6. InvalidConstructorError

Raised when constructor syntax in `<lvar>` declaration cannot be parsed.

In [9]:
# Invalid constructor syntax examples
invalid_constructors = [
    "<lvar .field name>value</lvar>",  # Missing model name
    "<lvar Model. name>value</lvar>",  # Missing field name
    "<lvar Model name>value</lvar>",  # Missing dot separator
    "<lvar 123Model.field>value</lvar>",  # Invalid identifier
]

print("Invalid constructor patterns:")
for invalid in invalid_constructors:
    print(f"  ❌ {invalid}")

print("\n✓ Valid pattern: <lvar ModelName.field_name optional_alias>value</lvar>")

Invalid constructor patterns:
  ❌ <lvar .field name>value</lvar>
  ❌ <lvar Model. name>value</lvar>
  ❌ <lvar Model name>value</lvar>
  ❌ <lvar 123Model.field>value</lvar>

✓ Valid pattern: <lvar ModelName.field_name optional_alias>value</lvar>


## 7. AmbiguousMatchError

Raised in fuzzy matching when multiple fields have similar similarity scores (tie scenario).

In [10]:
from lionpride.lndl.fuzzy import parse_lndl_fuzzy


# Model with similar field names
class Report(BaseModel):
    user_name: str
    username: str  # Very similar to user_name
    full_name: str


operable_fuzzy = Operable(
    specs=[
        Spec(str, name="user_name"),
        Spec(str, name="username"),
        Spec(str, name="full_name"),
    ]
)

# Response with ambiguous field name
ambiguous_response = """
OUT{
  user_name: "alice"
  usrname: "bob"      # Could match 'user_name' OR 'username' with similar scores
  full_name: "Alice Smith"
}
"""

try:
    # Fuzzy matching with default threshold
    parse_lndl_fuzzy(ambiguous_response, operable_fuzzy)
    print("✓ Fuzzy matching resolved ambiguous field")
except AmbiguousMatchError:
    print("✓ Caught AmbiguousMatchError in fuzzy matching")
    print("  Ambiguous field: usrname")
    print("  Could match: user_name OR username")
except Exception as e:
    # May not trigger ambiguity with current similarity algorithm
    print(f"Note: {type(e).__name__}: {e}")

Note: ParseError: Parse error at line 4, column 30: Expected COLON, got ID


## 8. Error Handling Patterns

Best practices for handling LNDL errors in production.

In [11]:
# Pattern 1: Catch all LNDL errors
def parse_with_logging(response: str, operable: Operable):
    """Parse with comprehensive LNDL error handling."""
    try:
        return parse_lndl(response, operable)
    except MissingOutBlockError:
        print("ERROR: LLM response missing OUT{} block")
        print("ACTION: Check system prompt includes LNDL instructions")
        raise
    except MissingFieldError as e:
        print(f"ERROR: Required field missing: {e}")
        print("ACTION: Review operable specs or add field to prompt")
        raise
    except TypeMismatchError as e:
        print(f"ERROR: Type mismatch: {e}")
        print("ACTION: Check lvar constructor matches operable spec")
        raise
    except LNDLError as e:
        # Catch any other LNDL-specific errors
        print(f"ERROR: LNDL parsing failed: {type(e).__name__}: {e}")
        raise


# Test with invalid response
operable_test = Operable(specs=[Spec(str, name="status")])
try:
    parse_with_logging("No OUT block here", operable_test)
except LNDLError:
    print("\n✓ Error logged and propagated")

ERROR: LLM response missing OUT{} block
ACTION: Check system prompt includes LNDL instructions

✓ Error logged and propagated


In [12]:
# Pattern 2: Retry with fuzzy matching on strict failure
def parse_with_fallback(response: str, operable: Operable):
    """Try strict parsing, fall back to fuzzy on field errors."""
    try:
        # First attempt: strict parsing
        return parse_lndl(response, operable)
    except (MissingFieldError, AmbiguousMatchError):
        print("⚠️  Strict parsing failed, trying fuzzy matching...")
        # Fallback: fuzzy matching with lenient threshold
        return parse_lndl_fuzzy(response, operable, threshold=0.75)
    except LNDLError:
        # Other LNDL errors can't be fixed by fuzzy matching
        raise


# Test with slightly malformed response
class Result(BaseModel):
    status: str


operable_fallback = Operable(specs=[Spec(str, name="status")])
fuzzy_response = """
OUT{
  statu: "success"  # Typo in field name
}
"""

try:
    _ = parse_with_fallback(fuzzy_response, operable_fallback)
    print("✓ Fuzzy matching would recover if 'statu' matched 'status'")
except Exception as e:
    print(f"Could not parse: {e}")

Could not parse: Parse error at line 3, column 27: Expected COLON, got ID


In [13]:
# Pattern 3: Validation pipeline with detailed diagnostics
def validate_lndl_response(response: str, operable: Operable) -> dict:
    """Validate LNDL response and return diagnostics."""
    diagnostics = {
        "valid": False,
        "stage": None,
        "error_type": None,
        "message": None,
    }

    try:
        diagnostics["stage"] = "out_block_extraction"
        # Use Lexer/Parser to check for OUT{} block
        lexer = Lexer(response)
        tokens = lexer.tokenize()
        parser = Parser(tokens, source_text=response)
        program = parser.parse()

        if not program.out_block:
            raise MissingOutBlockError("No OUT{} block found")

        diagnostics["stage"] = "parsing"
        _ = parse_lndl(response, operable)

        diagnostics["valid"] = True
        diagnostics["stage"] = "success"
        return diagnostics

    except MissingOutBlockError as e:
        diagnostics["error_type"] = "MissingOutBlockError"
        diagnostics["message"] = str(e)
    except MissingFieldError as e:
        diagnostics["error_type"] = "MissingFieldError"
        diagnostics["message"] = str(e)
    except TypeMismatchError as e:
        diagnostics["error_type"] = "TypeMismatchError"
        diagnostics["message"] = str(e)
    except LNDLError as e:
        diagnostics["error_type"] = type(e).__name__
        diagnostics["message"] = str(e)

    return diagnostics


# Test validation pipeline
test_operable = Operable(specs=[Spec(str, name="status")])
test_cases = [
    ("No OUT block", test_operable),
    ('OUT{ status: "success" }', test_operable),
]

for response, op in test_cases:
    diag = validate_lndl_response(response, op)
    status = "✓" if diag["valid"] else "✗"
    print(f"{status} Stage: {diag['stage']}, Error: {diag['error_type'] or 'None'}")

✗ Stage: out_block_extraction, Error: MissingOutBlockError
✓ Stage: success, Error: None


## 9. Error Prevention

Best practices to avoid LNDL errors.

In [14]:
# ✅ GOOD: Clear system prompt with examples
system_prompt = """
Return structured output using LNDL format:

OUT{
  field_name: "value"
  other_field: 123
}

Required fields: result, confidence
"""

# ✅ GOOD: Operable matches prompt exactly
operable_good = Operable(
    specs=[
        Spec(str, name="result", required=True),
        Spec(float, name="confidence", required=True),
    ]
)

print("✓ Clear alignment between prompt and operable")
print("  Required fields in prompt: result, confidence")
print(
    f"  Required specs in operable: {[s.name for s in operable_good.get_specs() if s.get('required', True)]}"
)

✓ Clear alignment between prompt and operable
  Required fields in prompt: result, confidence
  Required specs in operable: ['result', 'confidence']


In [15]:
# ✅ GOOD: Use fuzzy matching for user-facing applications
def robust_parse(response: str, operable: Operable):
    """Production-ready parsing with fuzzy fallback."""
    try:
        # Try strict first (fastest, most accurate)
        return parse_lndl(response, operable)
    except MissingFieldError:
        # Fuzzy can handle typos and near-misses
        return parse_lndl_fuzzy(response, operable, threshold=0.8)


print("✓ Robust parsing strategy:")
print("  1. Try strict parsing (fast path)")
print("  2. Fall back to fuzzy (handles typos)")
print("  3. Let other errors propagate (real issues)")

✓ Robust parsing strategy:
  1. Try strict parsing (fast path)
  2. Fall back to fuzzy (handles typos)
  3. Let other errors propagate (real issues)


In [16]:
# ✅ GOOD: Test error paths in development
def test_error_handling():
    """Verify error handling works as expected."""
    operable = Operable(specs=[Spec(str, name="status")])

    # Test 1: Missing OUT block
    try:
        parse_lndl("no block", operable)
        raise AssertionError("Should raise MissingOutBlockError")
    except MissingOutBlockError:
        print("✓ Test 1: MissingOutBlockError raised correctly")

    # Test 2: Missing required field
    try:
        parse_lndl('OUT{ wrong_field: "x" }', operable)
        raise AssertionError("Should raise MissingFieldError")
    except (MissingFieldError, ValueError):
        print("✓ Test 2: MissingFieldError raised correctly")

    print("\n✓ All error paths tested")


test_error_handling()

✓ Test 1: MissingOutBlockError raised correctly
✓ Test 2: MissingFieldError raised correctly

✓ All error paths tested


## Summary Checklist

**LNDL Error Types:**
- ✅ `LNDLError` - Base exception for all LNDL errors
- ✅ `MissingOutBlockError` - No OUT{} block or unbalanced braces
- ✅ `MissingFieldError` - Required Spec field missing from OUT{}
- ✅ `MissingLvarError` - OUT{} references non-existent lvar
- ✅ `TypeMismatchError` - Constructor class doesn't match Spec type
- ✅ `InvalidConstructorError` - Cannot parse constructor syntax
- ✅ `AmbiguousMatchError` - Multiple fields match with similar scores

**Error Handling Patterns:**
- ✅ Catch specific errors for targeted recovery
- ✅ Use fuzzy matching as fallback for field errors
- ✅ Build validation pipelines with diagnostics
- ✅ Test error paths in development

**Prevention:**
- ✅ Clear system prompts with LNDL examples
- ✅ Align operable specs with prompt requirements
- ✅ Use fuzzy matching for user-facing applications

**Next Steps:**
- See `parse_lndl` for strict parsing
- See `parse_lndl_fuzzy` for lenient parsing
- See `Operable` for spec definition